In [21]:
import requests, json
from time import sleep
import sys
import pandas as pd

In [2]:
# Secret token
token = open("ACCESS_TOKEN.txt", "r").read()

In [15]:
# number of artist ids already collected
OFFSET = !wc -l artists.txt
OFFSET = int(OFFSET[0].split()[0])

In [16]:
# constant values.
BASE_URL = "https://api.genius.com"
CLIENT_ACCESS_TOKEN = token
QUERY_SIZE=1000

In [17]:
# send request and get response in json format.
def _get(path, params=None, headers=None):

    # generate request URL
    requrl = '/'.join([BASE_URL, path])
    token = "Bearer {}".format(CLIENT_ACCESS_TOKEN)
    if headers:
        headers['Authorization'] = token
    else:
        headers = {"Authorization": token}

    response = requests.get(url=requrl, params=params, headers=headers)
    response.raise_for_status()

    return response.json()

In [18]:
# read in list of artist names
with open("artistNames.txt", "r") as f:
    artists = f.readlines()
    
artists = [x.strip() for x in artists]
print(str(len(artists))+" artists total")

12175 artists total


In [29]:
# query only a subset of artists at a time to not overwork server
# increment chunk_num on each run
artists = artists[OFFSET:]
artists_chunk = artists[0 : QUERY_SIZE]

# write (artist_name,artist_id) pairs to artists.txt file
with open("artists.txt", "a") as f:

    # iterate over artist names
    for i, artist in enumerate(artists_chunk):
        sys.stdout.write('\r' + str(i).zfill(4))

        # get artist_id
        find_id = _get("search", {'q': artist})
        
        artist_id = None
        for hit in find_id["response"]["hits"]:
            if hit["result"]["primary_artist"]:
                artist_id = hit["result"]["primary_artist"]["id"]
                break

        # write to file
        f.write(str(artist_id)+","+artist+"\n")

        sleep(.25)

0999